In [159]:
import requests
from time import sleep, strftime
import random
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
import shutil
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import pandas as pd
import datetime
import funciones_nba24
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from nba_api.stats import endpoints
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.cluster import DBSCAN 
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer,recall_score
from sklearn import tree

In [ ]:
#df_avanzadas.to_csv('estadisticas_tradicionales.csv', header=True, index=True)
#df_tradicional.to_csv('estadisticas_avanzadas.csv', header=True, index=True)

In [11]:
df = pd.read_csv("estadisticas_tradicionales.csv")

In [12]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [15]:
df.drop("PTS", axis=1, inplace=True)

In [67]:
df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,MIN,FGM,FGA,...,STL,BLK,TO,PF,PLUS_MINUS,local_visitante,gan_per,clase,acierto,aciertos2
0,22000490,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,31.833333,4.0,20.0,...,2.0,0.0,1.0,1.0,-15.0,V,P,0,True,False
1,22000490,1610612738,BOS,Boston,1628464,Daniel Theis,Daniel,11.333333,0.0,2.0,...,0.0,1.0,0.0,2.0,2.0,V,P,1,False,True
2,22000490,1610612738,BOS,Boston,202684,Tristan Thompson,Tristan,25.800000,5.0,10.0,...,1.0,0.0,0.0,2.0,-7.0,V,P,0,True,False
3,22000490,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,29.483333,6.0,16.0,...,1.0,0.0,2.0,1.0,-13.0,V,P,0,True,False
4,22000490,1610612738,BOS,Boston,201952,Jeff Teague,Jeff,28.566667,4.0,9.0,...,1.0,1.0,1.0,2.0,-22.0,V,P,0,True,False


# Kmeans

In [21]:
df.head().columns[7:26]

Index(['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF',
       'PLUS_MINUS'],
      dtype='object')

#### Escalar datos

In [26]:
X = MinMaxScaler().fit_transform(df.iloc[:, 7:26])
X.shape

(24693, 19)

#### Modelo kmeans

In [28]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
kmeans.labels_

array([0, 1, 0, ..., 0, 1, 1])

In [30]:
df["clase"]=kmeans.labels_

### Suponemos que clase 1 es ganador

In [48]:
df[:][["gan_per","clase"]].value_counts()


gan_per  clase
P        1        6477
G        1        6422
         0        6025
P        0        5769
dtype: int64

In [55]:
resul=[]
for i in range(len(df)): 
    if df["gan_per"][i]=="P" and df["clase"][i]==0:
        resul.append(True)
    elif df["gan_per"][i]=="P" and df["clase"][i]==1:
        resul.append(False)
    elif df["gan_per"][i]=="G" and df["clase"][i]==0:
        resul.append(False)
    elif df["gan_per"][i]=="G" and df["clase"][i]==1:
        resul.append(True)


In [58]:
df["acierto"]=resul

In [60]:
df["acierto"].value_counts()

False    12502
True     12191
Name: acierto, dtype: int64

#### Suponemos que 0 es ganador

In [63]:
resul2=[]
for i in range(len(df)): 
    if df["gan_per"][i]=="P" and df["clase"][i]==0:
        resul2.append(False)
    elif df["gan_per"][i]=="P" and df["clase"][i]==1:
        resul2.append(True)
    elif df["gan_per"][i]=="G" and df["clase"][i]==0:
        resul2.append(True)
    elif df["gan_per"][i]=="G" and df["clase"][i]==1:
        resul2.append(False)

In [64]:
df["aciertos2"]=resul2

In [65]:
df["aciertos2"].value_counts()

True     12502
False    12191
Name: aciertos2, dtype: int64

In [66]:
12502/(12502+12191)

0.5062973312274733

## Conclusiones 1:
- Kmeans con todas las variables numéricas no distingue entre ganadores y perdedores

In [75]:
df_kmeans = df.copy()

In [76]:
df_kmeans.drop(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME','FGM', 'FG_PCT','FG3M', 'OREB','BLK', 'FTM', 'clase', 'acierto', 'aciertos2' ], axis=1, inplace=True)

In [79]:
df_kmeans.columns[:13]

Index(['MIN', 'FGA', 'FG3A', 'FG3_PCT', 'FTA', 'FT_PCT', 'DREB', 'REB', 'AST',
       'STL', 'TO', 'PF', 'PLUS_MINUS'],
      dtype='object')

## Kmeans con las variables más importantes según eda

In [85]:
#Escalar datos
X = MinMaxScaler().fit_transform(df_kmeans.iloc[:, :13])
X.shape

#Modelo
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
kmeans.labels_

array([0, 1, 0, ..., 0, 1, 1])

In [86]:
df_kmeans["clase"]=kmeans.labels_

### Suponemos que 1 es ganador

In [87]:
resul=[]
for i in range(len(df)): 
    if df_kmeans["gan_per"][i]=="P" and df_kmeans["clase"][i]==0:
        resul.append(True)
    elif df_kmeans["gan_per"][i]=="P" and df_kmeans["clase"][i]==1:
        resul.append(False)
    elif df_kmeans["gan_per"][i]=="G" and df_kmeans["clase"][i]==0:
        resul.append(False)
    elif df_kmeans["gan_per"][i]=="G" and df_kmeans["clase"][i]==1:
        resul.append(True)

In [88]:
df_kmeans["aciertos"]=resul

In [89]:
df_kmeans["aciertos"].value_counts()

False    12459
True     12234
Name: aciertos, dtype: int64

In [90]:
12459/(12459+12234)

0.5045559470295226

In [91]:
df_kmeans.groupby("clase").mean()

,MIN,FGA,FG3A,FG3_PCT,FTA,FT_PCT,DREB,REB,AST,STL,TO,PF,PLUS_MINUS,aciertos
clase,,,,,,,,,,,,,,
0,27.925823,11.157943,4.051419,0.293272,3.929403,0.840443,4.149569,5.311029,3.137426,0.884348,1.653044,2.163973,0.971694,0.491081
1,17.551222,5.542425,2.475612,0.229993,0.281838,0.028538,2.346612,3.038974,1.521879,0.531681,0.827478,1.483375,-0.909896,0.499529


## Conclusiones 2:
- Kmeans con variables más importantes según eda no hay una agrupación clara entre ganadores y perdedores
- Es probable que se estén agrupando jugadores buenos de ambos equipos en una clase

# DBSCAN

In [97]:
df_dbscan = df_kmeans.copy()

In [101]:
df_dbscan.drop(["clase", "aciertos"], axis=1, inplace=True)

In [109]:
#Escalar datos
X = MinMaxScaler().fit_transform(df_dbscan.iloc[:, :13])
X.shape

epsilon = 0.3
minimumSamples = 5
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(X)
labels = db.labels_
labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [110]:
df_dbscan["clase"]=labels

In [111]:
df_dbscan["clase"].value_counts()

 0    22585
-1     2089
 3        5
 1        4
 4        4
 2        3
 5        3
Name: clase, dtype: int64

In [113]:
df_dbscan["gan_per"].value_counts()

G    12447
P    12246
Name: gan_per, dtype: int64

In [118]:
df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TO', 'PF', 'PLUS_MINUS', 'local_visitante', 'gan_per',
       'clase', 'acierto', 'aciertos2'],
      dtype='object')

## DBSCAN  no funciona, genera muchas clases

# Data Leakage

# Random Forest

In [122]:
a=["ATL", "BOS", "CLE", "NOP", "CHI", "DAL", "DEN", "GSW", "HOU", "LAC", "LAL", "MIA", "MIL", "MIN", "BKN", "NYK", "ORL", "IND",
"PHI", "PHX", "POR", "SAC", "SAS", "OKC", "TOR", "UTA", "MEM", "WAS", "DET", "CHA"]
combinaciones_games=[]
for i in itertools.combinations(a,2):
    combinaciones_games.append(i)
len(combinaciones_games)

435

In [ ]:
df_random_forest = df.copy()

## Label encoding

In [169]:
from sklearn.preprocessing import LabelEncoder

class_labelEncoding = LabelEncoder()
class_labelEncoding.fit(df["TEAM_ABBREVIATION"].values)
clase = class_labelEncoding.transform(df["TEAM_ABBREVIATION"].values)
clase


array([2, 2, 2, ..., 3, 3, 3])

In [170]:
df_random_forest["TEAM_ABBREVIATION"]=clase

In [171]:
class_labelEncoding = LabelEncoder()
class_labelEncoding.fit(df["PLAYER_NAME"].values)
clase = class_labelEncoding.transform(df["PLAYER_NAME"].values)
clase


array([249, 100, 507, ..., 137,  83,  37])

In [172]:
df_random_forest["PLAYER_NAME"]=clase

In [188]:
df_random_forest["local_visitante"]= df_random_forest["local_visitante"].map({"L":0, "V":1})

## Train Test: entrenamos hasta la combinación 304, índice :17548

### Averiguar el indice de la combinación 304

In [ ]:
c1="MIN"
c2="PHX"
df_enfrentamientos_equipo=pd.DataFrame()
for j in df["GAME_ID"].unique():
        df_partido = df[df["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])

In [374]:
df_enfrentamientos_equipo.iloc[-1:, :]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,MIN,FGM,FGA,...,STL,BLK,TO,PF,PLUS_MINUS,local_visitante,gan_per,clase,acierto,aciertos2
17547,22000526,1610612750,MIN,Minnesota,1627823,Juancho Hernangomez,Juancho,7.1,3.0,5.0,...,0.0,0.0,0.0,1.0,0.0,L,P,1,False,True


#### 'clase', 'acierto', 'aciertos2' vienen del kmeans

In [147]:
df_random_forest.drop(['GAME_ID', 'TEAM_ID', 'TEAM_CITY', 'PLAYER_ID', 'NICKNAME','FGM','FG3M', 'FTM',  'OREB',
                              'BLK', 'clase', 'acierto', 'aciertos2'], axis=1, inplace=True)

In [153]:
# nuestra x
x = df_random_forest.iloc[:, :-1]
#Nuestra y
y = df_random_forest.iloc[:, -1]

In [161]:
x.columns

Index(['TEAM_ABBREVIATION', 'PLAYER_NAME', 'MIN', 'FGA', 'FG_PCT', 'FG3A',
       'FG3_PCT', 'FTA', 'FT_PCT', 'DREB', 'REB', 'AST', 'STL', 'TO', 'PF',
       'PLUS_MINUS', 'local_visitante'],
      dtype='object')

### Xtrain y Train, xtest y test

In [190]:
x_train = x = df_random_forest.iloc[:17548, :-1]
y_train = df_random_forest.iloc[:17548, -1]
x_test = df_random_forest.iloc[17548:, :-1]
y_test = df_random_forest.iloc[17548:, -1]

7145

# Grid search 1

In [192]:
params={'n_estimators': [100,200,300],#Así se le asignan las posibilidades del valor que pueda tomat el parámetro
        'criterion':['gini','entropy'],
        #'max_depth': [3,4,5],# Maxima pofundidad del arbol
        #'max_features': [2, 3], # numero de features a considerar en cada split
        'max_leaf_nodes': [8], # maximo de nodos del arbol
        #'min_impurity_decrease' : [0.02,0.3], # un nuevo nodo se hará si al hacerse se decrece la impurity en un threshold por encima del valor
        'min_samples_split': [2,5] # The minimum number of samples required to split an internal node
        }


In [193]:
# creamos la GridSeacrh
#Esta celda como pa tomar desiciones, da las buenas combinaciones y los scores

scorers = {"f1_macro","accuracy","recall_macro"}#las métricas que quiero
clf = RandomForestClassifier()
grid_solver = GridSearchCV(estimator = clf, #el clasificador
                   param_grid = params, #el diccionario params
                   scoring=scorers,# las métricas
                   cv = 5,#hace k fold
                   refit="accuracy",#métrica pa mejorar, la que más importa
                   n_jobs=-1)

model_result = grid_solver.fit(x_train,y_train)

print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())
print(model_result.best_score_)
print(model_result.best_params_)

0.7128182373837458
0.7121429998847092
0.7126507823999987
0.715011135387311
{'criterion': 'entropy', 'max_leaf_nodes': 8, 'min_samples_split': 5, 'n_estimators': 200}


# Grid search 2

In [194]:
params={'n_estimators': [200,300,400],#Así se le asignan las posibilidades del valor que pueda tomat el parámetro
        'criterion':['gini','entropy'],
        #'max_depth': [3,4,5],# Maxima pofundidad del arbol
        #'max_features': [2, 3], # numero de features a considerar en cada split
        'max_leaf_nodes': [16], # maximo de nodos del arbol
        #'min_impurity_decrease' : [0.02,0.3], # un nuevo nodo se hará si al hacerse se decrece la impurity en un threshold por encima del valor
        'min_samples_split': [2,10] # The minimum number of samples required to split an internal node
        }

In [195]:
# creamos la GridSeacrh
#Esta celda como pa tomar desiciones, da las buenas combinaciones y los scores

scorers = {"f1_macro","accuracy","recall_macro"}#las métricas que quiero
clf = RandomForestClassifier()
grid_solver = GridSearchCV(estimator = clf, #el clasificador
                   param_grid = params, #el diccionario params
                   scoring=scorers,# las métricas
                   cv = 5,#hace k fold
                   refit="accuracy",#métrica pa mejorar, la que más importa
                   n_jobs=-1)

model_result = grid_solver.fit(x_train,y_train)

print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())
print(model_result.best_score_)
print(model_result.best_params_)

0.7169433402159999
0.7165272260698184
0.7168104808230201
0.7191135533455283
{'criterion': 'gini', 'max_leaf_nodes': 16, 'min_samples_split': 2, 'n_estimators': 400}


### Mejores parámetros random forest

In [ ]:
{'criterion': 'entropy', 'max_leaf_nodes': 8, 'min_samples_split': 5, 'n_estimators': 200}

In [198]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 200,
                             criterion= "entropy",
                             #max_depth= 5,
                             max_features= "auto",
                             max_leaf_nodes= 8,
                             min_impurity_decrease= 0.1,
                             min_samples_split= 5,
                             bootstrap=False,
                             random_state=123456789,
                             n_jobs= -1)

clf.fit(x_train, y_train)
yhat = clf.predict(x_test)
print("Accuracy: ", accuracy_score(y_test,yhat))


Accuracy:  0.6708187543736879


In [200]:
len(yhat)

7145

#### Agregar columnas reales y predichas al df de prueba el x_test

In [209]:
x_test["predicho"]=yhat
x_test["real"]=y_test


#### Copia del df original con las filas del x_test

In [216]:
df_random_test = df.iloc[17548:, :].copy()

In [218]:
df_random_test.drop(["clase", "acierto","aciertos2" ], axis=1, inplace=True)

In [222]:
df_random_test["predicho"]=yhat


In [343]:
len(df_random_test["GAME_ID"].unique())

336

### Generar listas por game_id de los equipos ganadores reales y predichos

In [340]:
valores_predichos=[]
valores_reales=[]

for j in df_random_test["GAME_ID"].unique():
    partido_pred = df_random_test[df_random_test["GAME_ID"]==int(j)][["TEAM_ABBREVIATION", "predicho"]].value_counts()
    
    equipos = df_random_test[df_random_test["GAME_ID"]==int(j)]["TEAM_ABBREVIATION"].unique()
    
    try:
        var1 = partido_pred[equipos[0]]["G"]
    except:
        var1=0
    try:
        var2 = partido_pred[equipos[1]]["G"]
    except:
        var2=0
    
    try:
        var3 = partido_pred[equipos[0]]["P"]
    except:
        var3=0
    
    try:
        var4 = partido_pred[equipos[1]]["P"]
    except:
        var4=0
        
        
    
    #comp2=partido_pred[equipos[0]]["P"] < partido_pred[equipos[1]]["P"]
    
    if (var1 > var2) and (var3 < var4):
         valores_predichos.append([j, equipos[0]])
    else:
        valores_predichos.append([j, equipos[1]])


    partido = df_random_test[df_random_test["GAME_ID"]==int(j)]
    valores_reales.append([j, partido[partido["gan_per"]=="G"]["TEAM_ABBREVIATION"].values[0]])

### Crear df con las listas por game_id con los ganadores reales y predichos

In [358]:
#valores_predichos[i][1]#nombres predichos
#valores_reales[i][1]# nombres reales
valores_reales[i][0]#id de juego

22000765

In [359]:
ids=[]
real=[]
prediccion=[]
for i in range(len(valores_reales)):
    ids.append(valores_reales[i][0])
    real.append(valores_reales[i][1])
    prediccion.append(valores_predichos[i][1])

In [360]:
df_aciertos_random_forest = pd.DataFrame({"ID":ids, "ganadores_reales":real, "ganadores_predichos":prediccion})
df_aciertos_random_forest

,ID,ganadores_reales,ganadores_predichos
0,22000599,MIN,MIN
1,22000591,POR,POR
2,22000122,POR,POR
3,22000891,SAC,MIN
4,22000880,MIN,MIN
...,...,...,...
331,22000718,CHA,CHA
332,22000215,CHA,CHA
333,22000984,CHA,CHA
334,22000957,CHA,CHA


### Crear columna con aciertos y desaciertos, revisar porcentaje de aciertos

In [364]:
aciertos_r=[]
for i in range(len(df_aciertos_random_forest)):
    if df_aciertos_random_forest["ganadores_reales"][i] == df_aciertos_random_forest["ganadores_predichos"][i]:
        aciertos_r.append(True)
    else:
        aciertos_r.append(False)
df_aciertos_random_forest["aciertos"]=aciertos_r

In [366]:
df_aciertos_random_forest["aciertos"].value_counts()

True     307
False     29
Name: aciertos, dtype: int64

## Porcentaje de acierto

In [367]:
307/(29+307)

0.9136904761904762

In [383]:
df.iloc[17548:, :]

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,MIN,FGM,FGA,...,STL,BLK,TO,PF,PLUS_MINUS,local_visitante,gan_per,clase,acierto,aciertos2
17548,22000599,1610612757,POR,Portland,1627884,Derrick Jones Jr.,Derrick,19.833333,0.0,4.0,...,1.0,0.0,0.0,2.0,-11.0,V,P,1,False,True
17549,22000599,1610612757,POR,Portland,203496,Robert Covington,Robert,38.533333,7.0,13.0,...,1.0,1.0,2.0,5.0,4.0,V,P,0,True,False
17550,22000599,1610612757,POR,Portland,202683,Enes Kanter,Enes,26.083333,2.0,8.0,...,1.0,0.0,2.0,4.0,-11.0,V,P,1,False,True
17551,22000599,1610612757,POR,Portland,1629018,Gary Trent Jr.,Gary,33.800000,6.0,14.0,...,2.0,0.0,0.0,2.0,1.0,V,P,0,True,False
17552,22000599,1610612757,POR,Portland,203081,Damian Lillard,Damian,37.133333,12.0,21.0,...,0.0,1.0,5.0,5.0,-2.0,V,P,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24688,22000568,1610612766,CHA,Charlotte,1628970,Miles Bridges,Miles,23.400000,3.0,6.0,...,0.0,0.0,0.0,1.0,-4.0,L,G,1,True,False
24689,22000568,1610612766,CHA,Charlotte,1628370,Malik Monk,Malik,17.466667,4.0,11.0,...,0.0,0.0,0.0,1.0,1.0,L,G,1,True,False
24690,22000568,1610612766,CHA,Charlotte,1628984,Devonte' Graham,Devonte',18.483333,2.0,7.0,...,1.0,0.0,0.0,0.0,2.0,L,G,0,False,True
24691,22000568,1610612766,CHA,Charlotte,1628998,Cody Martin,Cody,10.066667,2.0,3.0,...,0.0,0.0,0.0,3.0,-3.0,L,G,1,True,False


In [385]:
c1="POR"
c2="MIN"
df_enfrentamientos_equipo=pd.DataFrame()
for j in df["GAME_ID"].unique():
        df_partido = df[df["GAME_ID"]==int(j)]
        if (df_partido["TEAM_ABBREVIATION"].unique()[0]==c1 or df_partido["TEAM_ABBREVIATION"].unique()[0]==c2 ) and (df_partido["TEAM_ABBREVIATION"].unique()[1]==c2 or df_partido["TEAM_ABBREVIATION"].unique()[1]==c1):
            df_enfrentamientos_equipo = pd.concat([df_enfrentamientos_equipo, df_partido])

In [391]:
df_enfrentamientos_equipo[df_enfrentamientos_equipo["GAME_ID"].isin([22000599, 22000591])].groupby("PLAYER_NAME")

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,MIN,FGM,FGA,...,STL,BLK,TO,PF,PLUS_MINUS,local_visitante,gan_per,clase,acierto,aciertos2
17548,22000599,1610612757,POR,Portland,1627884,Derrick Jones Jr.,Derrick,19.833333,0.0,4.0,...,1.0,0.0,0.0,2.0,-11.0,V,P,1,False,True
17549,22000599,1610612757,POR,Portland,203496,Robert Covington,Robert,38.533333,7.0,13.0,...,1.0,1.0,2.0,5.0,4.0,V,P,0,True,False
17550,22000599,1610612757,POR,Portland,202683,Enes Kanter,Enes,26.083333,2.0,8.0,...,1.0,0.0,2.0,4.0,-11.0,V,P,1,False,True
17551,22000599,1610612757,POR,Portland,1629018,Gary Trent Jr.,Gary,33.800000,6.0,14.0,...,2.0,0.0,0.0,2.0,1.0,V,P,0,True,False
17552,22000599,1610612757,POR,Portland,203081,Damian Lillard,Damian,37.133333,12.0,21.0,...,0.0,1.0,5.0,5.0,-2.0,V,P,0,True,False
17553,22000599,1610612757,POR,Portland,2546,Carmelo Anthony,Carmelo,29.516667,6.0,15.0,...,0.0,1.0,1.0,3.0,8.0,V,P,0,True,False
17554,22000599,1610612757,POR,Portland,1629642,Nassir Little,Nassir,19.366667,2.0,5.0,...,0.0,0.0,1.0,2.0,-1.0,V,P,1,False,True
17555,22000599,1610612757,POR,Portland,203918,Rodney Hood,Rodney,24.583333,2.0,5.0,...,0.0,0.0,0.0,5.0,2.0,V,P,1,False,True
17556,22000599,1610612757,POR,Portland,1629014,Anfernee Simons,Anfernee,11.150000,2.0,4.0,...,1.0,0.0,0.0,1.0,0.0,V,P,1,False,True
17557,22000599,1610612750,MIN,Minnesota,1627774,Jake Layman,Jake,20.533333,2.0,4.0,...,0.0,0.0,1.0,1.0,10.0,L,G,1,True,False


In [392]:
x_train

,TEAM_ABBREVIATION,PLAYER_NAME,MIN,FGA,FG_PCT,FG3A,FG3_PCT,FTA,FT_PCT,DREB,REB,AST,STL,TO,PF,PLUS_MINUS,local_visitante
0,2,249,31.833333,20.0,0.200,8.0,0.125,4.0,1.00,0.0,1.0,1.0,2.0,1.0,1.0,-15.0,1
1,2,100,11.333333,2.0,0.000,1.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1
2,2,507,25.800000,10.0,0.500,0.0,0.000,4.0,0.75,5.0,13.0,0.0,1.0,0.0,2.0,-7.0,1
3,2,246,29.483333,16.0,0.375,6.0,0.000,5.0,1.00,3.0,4.0,5.0,1.0,2.0,1.0,-13.0,1
4,2,251,28.566667,9.0,0.444,1.0,0.000,6.0,1.00,5.0,5.0,5.0,1.0,1.0,2.0,-22.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17543,17,269,22.783333,7.0,0.286,4.0,0.500,0.0,0.00,2.0,4.0,4.0,1.0,0.0,1.0,-7.0,0
17544,17,390,13.400000,6.0,0.667,1.0,0.000,2.0,0.50,1.0,2.0,2.0,0.0,0.0,5.0,-9.0,0
17545,17,241,9.100000,3.0,0.000,2.0,0.000,0.0,0.00,2.0,2.0,1.0,1.0,0.0,0.0,-6.0,0
17546,17,217,8.750000,3.0,0.667,2.0,0.500,0.0,0.00,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0
